In [1]:
import cv2
import math
import pandas as pd
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 500)
import numpy as np
from scipy.stats import linregress
from bokeh.models import Row
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, WheelZoomTool, NumeralTickFormatter
from bokeh.plotting import figure, show
output_notebook()

Loading BokehJS ...

Example of video use for the script.  

CPT Middle East TOP 8 Round 1 Game 1 of Angrybird vs Big Bird

<video width="858" height="480" 
       src="Angrybird vs Bigbird G1R1.mp4"  
       controls>
</video>

function that measures the edges using findContours in opencv

In [2]:
def display_health_percentage(framein, edgesin, x1, x2, y1, y2, x_axis, y_axis):
    crop_img = framein[x1:x2, y1:y2]
    crop_thresh_1 = edgesin[x1:x2, y1:y2]
    # finding lines and 
    cnts = cv2.findContours(crop_thresh_1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    if len(cnts) >= 1:
        for c in cnts:
            c = cnts[-1] # getting last of list
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(crop_img, (x, y), (x + w, y + h), (36, 255, 12), 2)
            width_text = cv2.putText(frame, '{:.0%}'.format(round(w) / 680),
                                     (x_axis, y_axis), cv2.FONT_HERSHEY_SIMPLEX, 1,
                                     (255, 255, 255), 2, cv2.LINE_AA, False)
        output = round(w) / 680 # 680 is the lenght of the line; to calculate % 
    else:
        output = 'null'
    return output

loop that goes through each frame of the video and does edge detection and such

In [3]:
cap = cv2.VideoCapture('Angrybird vs Bigbird G1R1.mp4') # enter the video path
# Properties for writing the video; makes script slow
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# fps = cap.get(cv2.CAP_PROP_FPS)

# video_writer = cv2.VideoWriter('frame.mp4', cv2.VideoWriter_fourcc(*'FMP4'),
#                                fps, (width, height), isColor=True)
# list creation
framelist = []
player1hp = []
player2hp = []
df = pd.DataFrame()

i = 0  # frame counter
frameTime = 1  # time of each frame in ms, you can add logic to change this value.
while cap.isOpened():
    ret = cap.grab()  # grab frame
    if ret:
        i = i + 1  # increment counter
        if i % 1 == 0:  # display only one third of the frames, you can change this parameter according to your needs
            ret, frame = cap.retrieve()  # decode frame
            # used for player hp measure
            crop_frame = frame[0:150, 0:1900]
            l_b = np.array([0, 0, 0])
            u_b = np.array([179, 255, 255])
            
            hsv = cv2.cvtColor(crop_frame, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv, l_b, u_b)
            
            res = cv2.bitwise_and(crop_frame, crop_frame, mask=mask)
            gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)

            # apply hsv color filter to add mask and get edges
            kernel_size = 5
            blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
            cv2.imshow("blur_gray", blur_gray)
            #controls edge threshold 
            low_threshold = 50
            high_threshold = 230
            edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
            cv2.imshow("edges", edges)
            # get health info
            player1health = display_health_percentage(frame, edges, 95, 101, 188, 872, 20, 88)
            player2health = display_health_percentage(frame, edges, 95, 101, 1047, 1726, 1780, 88)
            cv2.imshow("frame", frame)
            framelist.append(i)
            player1hp.append(player1health)
            player2hp.append(player2health)
            
#             video_writer.write(frame)

        if cv2.waitKey(frameTime) & 0xFF == ord('q'):
            break

        frameId = cap.get(1)  # current frame number
    else:
        break
        
cap.release()
cv2.destroyAllWindows()
# video_writer.release()
df['frame'] = framelist
df['player_1_hp'] = player1hp
df['player_2_hp'] = player2hp
df.to_csv('data.csv')

runtime of script for just health measure

<video width="858" height="480" 
       src="script runtime.mkv"  
       controls>
</video>

dataframe of health values from video

In [4]:
df

,frame,player_1_hp,player_2_hp
0,1,0.007353,0.014706
1,2,0.007353,0.014706
2,3,0.007353,0.014706
3,4,0.007353,0.014706
4,5,0.007353,0.014706
...,...,...,...
3718,3719,0.529412,null
3719,3720,0.014706,null
3720,3721,0.014706,null
3721,3722,0.529412,null


graph of the raw data

In [5]:
source = ColumnDataSource(df)

subset= df[(df['player_1_hp'] != 'null') & (df['player_2_hp'] != 'null')].copy()
subset['player_1_hp'] = subset['player_1_hp'].astype(float)
subset['player_2_hp'] = subset['player_2_hp'].astype(float)

p = figure(title="Raw Players HP Values", width=900, height=500, tools="", toolbar_location=None)
p1 = p.line(x='frame', y='player_1_hp',source=source, legend_label="Player 1 Health",
       line_color='#bd0052')
res = linregress(subset['frame'], subset['player_1_hp'])
y_regress = res.slope * subset['frame'] + res.intercept
p.line(x=subset['frame'], y=y_regress, color='#bd0052', legend_label="Player 1 Health")



p.add_tools(HoverTool(tooltips=[("Player 1 Health", "@player_1_hp")],
                      toggleable=False, renderers=[p1]))

p2 = p.line(x='frame', y='player_2_hp',source=source, legend_label="Player 2 Health",
       line_color='#1c63bb')
res = linregress(subset['frame'], subset['player_2_hp'])
y_regress = res.slope * subset['frame'] + res.intercept
p.line(x=subset['frame'], y=y_regress, color='#1c63bb', legend_label="Player 2 Health")

p.add_tools(HoverTool(tooltips=[("Player 2 Health", "@player_2_hp")],
                      toggleable=False, renderers=[p2]))

p.toolbar.active_scroll = p.select_one(WheelZoomTool)
p.xaxis.axis_label = 'Frames'
p.yaxis.axis_label = 'Health Percentage'
p.yaxis.formatter = NumeralTickFormatter(format='0 %')
p.legend.location = "top_right"
p.legend.click_policy="hide"

show(p)